In [1]:
import os
import json
import math

import sys
sys.path.append("../utils")
from minimal_image import get_compact

from skimage.io import MultiImage

import matplotlib.pyplot as plt

import numpy as np

import cv2 as cv

from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [2]:
with open("../input/compact_representation.json", "r") as file:
    compact_representation = json.load(file)

In [3]:
data_dir = "../input/prostate-cancer-grade-assessment/train_images/"

In [4]:
# class Data(Dataset):
#     def __init__(self):
#         self.keys = list(compact_representation.keys())
    
#     def __len__(self):
#         return len(compact_representation)
    
#     def __getitem__(self, item):
#         key = self.keys[item]
#         item = compact_representation[key]
        
#         img = MultiImage(os.path.join(data_dir, key))[1]
#         compact = get_compact(img, item)
        
#         return img.shape, compact.shape

In [5]:
# loader = DataLoader(Data(), batch_size=1, num_workers=0)

# orig_shapes = []
# compact_shapes = []

# for orig, comp in tqdm(loader):
#     orig_shapes.append(orig)
#     compact_shapes.append(comp)

In [6]:
# import torch

In [7]:
# torch.stack(orig_shapes)

In [8]:
# orig_shapes = torch.cat([torch.stack(i, -1) for i in orig_shapes])


In [9]:
# compact_shapes = torch.cat([torch.stack(i, -1) for i in compact_shapes])

In [10]:
# orig_shapes.shape, compact_shapes.shape

In [11]:
# orig_shapes = orig_shapes.numpy()
# compact_shapes = compact_shapes.numpy()

In [12]:
import pandas as pd

In [13]:
# df = pd.DataFrame({
#     "names": list(compact_representation.keys()),
#     "h": orig_shapes[:, 0],
#     "w": orig_shapes[:, 1],
#     "comp_h": compact_shapes[:, 0],
#     "comp_w": compact_shapes[:, 1],
# })

In [14]:
# df

In [15]:
# df.to_csv("shapes.csv", index=False)

In [16]:
df = pd.read_csv("shapes.csv")
df

,names,h,w,comp_h,comp_w
0,2f305c85a7bb5ce4ea89ceee84f17f6b.tiff,7568,5727,775,6276
1,5d3dfae104348251b8fbc99f8d39a601.tiff,3008,960,896,2944
2,89f418346fc412a6458aaa05a224af3a.tiff,3392,2112,1858,3509
3,b53d843700d9665439ec8791e4024d70.tiff,2752,3072,1447,3748
4,0005f7aaab2800f6170c399693a96917.tiff,7360,6912,1313,5382
...,...,...,...,...,...
10611,ffd2841373b39792ab0c84cccd066e31.tiff,8192,2944,2900,6992
10612,ffdc59cd580a1468eac0e6a32dd1ff2d.tiff,1280,3584,878,3572
10613,ffe06afd66a93258f8fabdef6044e181.tiff,7168,2496,1988,7407
10614,ffe236a25d4cbed59438220799920749.tiff,2688,6656,1224,6988


In [17]:
df['h'] = df['h'] / 4
df['w'] = df['w'] / 4
df['comp_h'] = df['comp_h'] / 4
df['comp_w'] = df['comp_w'] / 4

In [18]:
import seaborn as sns

In [19]:
[df['h'].max(), df['comp_h'].max()], [df['h'].min(), df['comp_h'].min()]

([3456.0, 2055.25], [112.0, 0.0])

In [20]:
[df['w'].max(), df['comp_w'].max()], [df['w'].min(), df['comp_w'].min()]

([6144.0, 3531.75], [80.0, 0.0])

In [21]:
masks_dir = "../input/prostate-cancer-grade-assessment/train_label_masks"

In [22]:
df['comp_w'].argmax()

6007

In [129]:
import cv2 as cv

In [274]:
def tile(img, sz=256, N=100):
    shape = img.shape
    pad0, pad1 = (sz - shape[0] % sz) % sz, (sz - shape[1] % sz) % sz

    img = np.pad(img, [[pad0 // 2, pad0 - pad0 // 2], [pad1 // 2, pad1 - pad1 // 2], [0, 0]],
                 constant_values=255)
    
    result = img
    shape = img.shape

    img = img.reshape(img.shape[0] // sz, sz, img.shape[1] // sz, sz, 3)
    img = img.transpose(0, 2, 1, 3, 4).reshape(-1, sz, sz, 3)
    
    num = len(img)

    if len(img) < N:
        img = np.pad(img, [[0, N - len(img)], [0, 0], [0, 0], [0, 0]], constant_values=255)

    idxs = np.argsort(img.reshape(img.shape[0], -1).sum(-1))[:N]

    img = img[idxs]
    s = int(np.sqrt(N))
    r = np.full([sz * s, sz * s, 3], 255, dtype=np.uint8)

    indexes = np.arange(N)

    for i, j in enumerate(indexes):
        if i >= len(img):
            break
        if i >= num:
            break
            
        x = j % s
        y = j // s
        img_i = img[i]
        if r.dtype != img_i.dtype:
            result = result.astype(img_i.dtype)
        r[y * sz:(y + 1) * sz, x * sz:(x + 1) * sz] = img_i
        
        x = idxs[i] % (shape[1] // sz)
        y = idxs[i] // (shape[1] // sz)
        x1 = x * sz
        x2 = (x + 1) * sz
        y1 = y * sz
        y2 = (y + 1) * sz
        cv.rectangle(result, (x1, y1), (x2, y2), (0, 255, 0), 2)


    return result, r

In [270]:
(df["comp_h"] * df['comp_w']).argmax()

6831

In [277]:
idx = 0
index = 6831

for key, item in compact_representation.items():
    key = df.iloc[index]["names"]
    item = compact_representation[key]
    img = MultiImage(os.path.join(data_dir, key))[idx]
#     mask = MultiImage(os.path.join(masks_dir, os.path.splitext(key)[0] + "_mask.tiff"))[idx]
#     img = get_compact(img, item)
    img, img_tile = tile(img)
#     mask = get_compact(mask, item, default_val=0)
    
    break

plt.figure(figsize=(30, 15))
# plt.subplot(211)
plt.imshow(img)
# plt.subplot(212)
# plt.imshow(img_tile)
# plt.subplot(212)
# plt.imshow((mask > 1) * 255);

KeyboardInterrupt: 

In [264]:
np.unique(mask)

array([0, 1], dtype=uint8)

In [37]:
df_train = pd.read_csv("../input/prostate-cancer-grade-assessment/train.csv")

In [101]:
df_train[df_train['image_id'] == key.replace(".tiff", "")]

,image_id,data_provider,isup_grade,gleason_score
351,0905d7115955cdd6f438a64bf5061902,karolinska,3,4+3


In [47]:
mask.max()

3

In [103]:
df_train.groupby("data_provider").count()

,image_id,isup_grade,gleason_score
data_provider,,,
karolinska,5456,5456,5456
radboud,5160,5160,5160
